# Accessing the project directory on my Google Drive  

In [1]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)
project_dir = "/drive/My Drive/RNNSeq2Seq/"

# change working directory to project_dir
os.chdir(project_dir)

Mounted at /drive


# Dependencies

In [2]:
from scripts.model import *
from scripts.dataloader import create_dataloader
from scripts.utils import *
from scripts.pytorch_utils import *
import matplotlib.pyplot as plt

# Helper function to help visualize 

In [3]:
def plot_log(log):
    len_range = len(log) -1 if "Best eval accu" in log else len(log)
    epoch_nums, train_loss, dev_loss = [], [], []
    train_acc, dev_acc = [], []

    for epoch in range(1, len_range+1):
        epoch_nums.append(epoch)
        train = log[f"Epoch#{epoch}"]["Train"]
        dev = log[f"Epoch#{epoch}"]["Eval"]
        train_loss.append(train['loss'])
        train_acc.append(train['abosulate accuracy'])

        dev_loss.append(dev['loss'])
        dev_acc.append(dev['abosulate accuracy'])

    ax1 = plt.subplot(211)
    ax1.plot(epoch_nums, train_loss, label="train loss")
    ax1.plot(epoch_nums, dev_loss, label="dev loss")
    ax1.legend()

    ax2 = plt.subplot(212)
    ax2.plot(epoch_nums, train_acc, label="train acc")
    ax2.plot(epoch_nums, dev_acc, label="dev acc")
    ax2.legend()
    plt.show()

In [4]:
folder = "tuning_data/"
experiment_num = 6
shuffle_batch = True

# must be the num of seq per seq length if shuffle_batch is False
# this "tr_batch_size" will be used for dev, and shuffled test sets
tr_batch_size = 250 
dev_batch_size = 500 if not shuffle_batch else tr_batch_size

In [6]:
# setups for training and evaluting

print_freq = 5
acc_threshold = 0.5 # for saving the best trained models 
max_epoch_num = 150 # max num of epoches 
train_acc_exit=0.92 # train acc exit threshold (eval_acc_exit must also be met)
eval_acc_exit=0.92 # dev acc exit threshold (train_acc_exit must also be met)
teacher_forcing_ratio = 1.0 # probability of using real symbol from target sequence for training

# RevStr

In [7]:
train = read_data(join(folder, "RevStr/train.txt"))
dev = read_data(join(folder, "RevStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1

In [8]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_vocab_size = len(in_vocab) + 2
out_vocab_size = len(out_vocab) + 2

hidden_size = 512
embd_dim = 256
num_layers = 1
rnn_type = "GRU"
dropout_rate = 0.0
bidirectional = True
use_attention = False
reduction_method = torch.sum

learning_rate = 5e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [9]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 1.9237878412008285, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.002476666696020402, 'overlap rate': 0.4378767117857933}
evaluation performance: {'loss': 2.3328290075063705, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0029357627499848604, 'overlap rate': 0.4429900150746107}

Current epoch: 10, 
training performance: {'loss': 1.7694720864295959, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.005028333427617326, 'overlap rate': 0.4748150482773781}
evaluation performance: {'loss': 1.8242515951395035, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0051141667441697795, 'overlap rate': 0.47311587892472745}

Current epoch: 15, 
training performance: {'loss': 1.4525754064321519, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.010956666781567037, 'overlap rate': 0.545226725935936}
evaluation performance: {'loss': 1.6563551649451256, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.011

# RedStr



In [10]:
train = read_data(join(folder, "RedStr/train.txt"))
dev = read_data(join(folder, "RedStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1


In [11]:
hidden_size = 512
embd_dim = 256
num_layers = 1
rnn_type = "GRU"
encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

learning_rate = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 1.7317734241485596, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.005340000102296472, 'overlap rate': 0.4814817138016224}
evaluation performance: {'loss': 1.9603688269853592, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.005284166743513197, 'overlap rate': 0.479718379303813}

Current epoch: 10, 
training performance: {'loss': 1.4587057948112487, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.00951000014320016, 'overlap rate': 0.541305062174797}
evaluation performance: {'loss': 1.6736969023942947, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.01021166676073335, 'overlap rate': 0.48298921398818495}

Current epoch: 15, 
training performance: {'loss': 1.1294326335191727, 'abosulate accuracy': 0.002600000100210309, 'consecutive overlap rate': 0.024900000402703883, 'overlap rate': 0.6279183939099312}
evaluation performance: {'loss': 1.6371349662542343, 'abosulate accuracy': 0.001500000071246177, 'con

## Hyperparameter set 2

In [15]:
hidden_size = 512
embd_dim = 256
num_layers = 1
rnn_type = "GRU"
encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 1.9470311462879182, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.007613333419431001, 'overlap rate': 0.44300004467368126}
evaluation performance: {'loss': 2.753030413389206, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0067333334125578405, 'overlap rate': 0.44508671425282953}

Current epoch: 10, 
training performance: {'loss': 1.9266727149486542, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.005895000114105642, 'overlap rate': 0.44863004609942436}
evaluation performance: {'loss': 2.1192882999777796, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.003926666712504812, 'overlap rate': 0.39697420708835124}

Current epoch: 15, 
training performance: {'loss': 1.84060760140419, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.0060566667234525084, 'overlap rate': 0.4578050471842289}
evaluation performance: {'loss': 1.9448771730065346, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.00